# MiniBatchKMeans Demo
This demo illustrates how SIMBSIG can be used for MiniBatchKMeans, and how the use compares to scikit-learn. 

In [ ]:
!pip install simbsig

In [ ]:
from simbsig.cluster import MiniBatchKMeans
from sklearn.cluster import MiniBatchKMeans as MiniBatchKMeans_sk
import h5py as h5py
import numpy as np
from sklearn.datasets import make_blobs

## Set Parameters

In [ ]:
n_clusters = 3
trn_pts_per_cluster = 10
query_pts_per_cluster = 5
n_dim = 2
rng = np.random.RandomState(42)
centers = rng.uniform(low=-20,high=20,size=(n_clusters,n_dim))
centers_init = rng.uniform(low=-20,high=20,size=(n_clusters,n_dim))

## Sort cluster centers by first coordinate

In [ ]:
def sort_centers(centers):
    sorted_idxs = np.argsort(centers[:,0],axis=0)
    return centers[sorted_idxs]

## Create Toy Data
### numpy arrays

In [ ]:
X_train, y_train = make_blobs(n_samples=n_clusters * trn_pts_per_cluster, centers=centers, n_features=n_dim,
                              random_state=42)

X_query, y_query = make_blobs(n_samples=n_clusters * query_pts_per_cluster, centers=centers, n_features=n_dim,
                              random_state=43)

### hdf5 files
If you work from your computer, you can execute the statement below:
Of course you can also store the data in a directory more suitable to you.

In [ ]:
# when working from your computer, your disk can be used to save and read data
import os
dataset_path = os.path.dirname(os.path.realpath("__file__"))

In [ ]:
# hdf5 files using h5py
train_file = f'train.hdf5'
query_file = f'query.hdf5'

with h5py.File(os.path.join(dataset_path, f"{train_file}"), 'w') as f:
    f.create_dataset("X", data=X_train)

with h5py.File(os.path.join(dataset_path, f"{query_file}"), 'w') as f:
    f.create_dataset("X", data=X_query)

## Scikit-Learn


In [ ]:
mbkm_sk = MiniBatchKMeans_sk(n_clusters=3, init=centers_init, n_init=1,random_state=42)

mbkm_sk.fit(X_train)
mbkm_sk.cluster_centers_ = sort_centers(mbkm_sk.cluster_centers_)
mbkm_sk.predict(X_query)

## SIMBSIG
### Using numpy arrays and CPU only
SIMBSIG can be used very similar to scikit-learn. In an existing workflow using scikit-learn, which may be on the verge of exceeding runtime or memory requirements, this allows a seamless transition to SIMBSIG. Notice that when using different methods for KMeans clustering, the labels assigned to the clusters may be of different order.


In [ ]:
mbkm = MiniBatchKMeans(n_clusters=3, init=centers_init, random_state=42)

mbkm.fit(X_train)
mbkm.cluster_centers_ = sort_centers(mbkm.cluster_centers_)
mbkm.predict(X_query)

### Using hdf5 files and CPU only
If saving the entire data at once in the computer memory using numpy arrays is not reasonable anymore, the hdf5 file format can help. SIMBSIG can use data in hdf5 files, by setting the `mode` argument to `cpu`

In [ ]:
mbkm_hdf5 = MiniBatchKMeans(n_clusters=3, init=centers_init, mode='hdf5', random_state=42)

# open the hdf5 file for use
train_data = h5py.File(os.path.join(dataset_path, train_file), 'r')
query_data = h5py.File(os.path.join(dataset_path, query_file))

mbkm_hdf5.fit(train_data)
mbkm_hdf5.cluster_centers_ = sort_centers(mbkm_hdf5.cluster_centers_)
pred = mbkm_hdf5.predict(query_data)

# close hdf5 files
train_data.close()
query_data.close()

pred

### Using GPU acceleration
If data gets big, the execution time becomes an issue. SIMBSIG features GPU acceleration, by setting the `device` argument to `gpu`. This works with both inputs, numpy arrays and hdf5 files.

In [ ]:
mbkm_hdf5_gpu = MiniBatchKMeans(n_neighbors=2, mode='hdf5', device='gpu', tol=1e-8, max_iter=500, init=centers_init, random_state=42)

# open the hdf5 file for use
train_data = h5py.File(os.path.join(dataset_path, train_file), 'r')
query_data = h5py.File(os.path.join(dataset_path, query_file))

mbkm_hdf5_gpu.fit(train_data)
mbkm_hdf5_gpu.cluster_centers_ = sort_centers(mbkm_hdf5_gpu.cluster_centers_)
pred = mbkm_hdf5_gpu.predict(query_data)

# close hdf5 files
train_data.close()
query_data.close()

pred